Runeberg
========

In [1]:
from everything import *
from dasem.runeberg import Runeberg

In [2]:
runeberg = Runeberg()

In [3]:
catalogue = runeberg.catalogue()

In [4]:
catalogue.ix[catalogue.language == 'dk', :].head(15)

,author,author_id,book_id,language,title,type,year
19,Alexander von Humboldt,humboale,eurasrus,dk,A. v. Humboldts Reiser i det Europæiske og Asi...,Book,1856
25,Hans Christian Ørsted,orstehan,aanden,dk,Aanden i Naturen,Book,1850
27,Karl Gjellerup,gjelleka,aanderotid,dk,Aander og Tider. Et Rekviem over Charles Darwin,Book,1882
28,Frederik Dreier,dreiefre,aandetroen,dk,Aandetroen og Den frie Tænkning,Book,1852
29,Emil Aarestrup,aarestru,aarestrups,dk,Aarestrups samlede digte,Book,1913
38,Adam Bremensis,adambrem,adamafbrem,dk,Adam af Bremen om Menigheden i Norden under Er...,Book,1862
39,Gustav Wied,wiedgust,adelgejs,dk,"Adel, Gejstlighed, Borger og Bonde",Book,1897
73,Christian Frederik Emil Horneman,hornechr,aladdinev,dk,Aladdin. Eventyr-Opera i 4e Akter,Music,1893
83,Jens Zetlitz,zetlijen,alkoran,dk,Alkoran d. e. Capitlernes Bog for 1806,Book,1806
96,Thomas Hansen Erslew,ersletho,almiforf,dk,Almindeligt Forfatter-Lexicon for Kongeriget D...,Book,1843-1868


In [5]:
query = """
SELECT ?author ?authorLabel ?runeberg WHERE {
  ?author wdt:P3154 ?runeberg .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "da". }  
} 
"""
service = sparql.Service('https://query.wikidata.org/sparql', method="GET")
response = service.query(query)
wikidata = pd.DataFrame(response.fetchall(), columns=response.variables)

In [6]:
wikidata.head()

,author,authorLabel,runeberg
0,http://www.wikidata.org/entity/Q538109,Ture Nerman,nermatur
1,http://www.wikidata.org/entity/Q4722812,Alfred Hedenstierna,hedensal
2,http://www.wikidata.org/entity/Q5577698,Johan Gustaf Björklund,bjorkljg
3,http://www.wikidata.org/entity/Q6141019,Ludvig Jönsson,jonsslud
4,http://www.wikidata.org/entity/Q4753809,Anders Henrikson,henriand


In [7]:
wikidata.runeberg = wikidata.runeberg.astype(str)
wikidata.author = wikidata.author.astype(str)
wikidata.authorLabel = wikidata.authorLabel.astype(unicode)
wikidata.rename(columns={'runeberg': 'author_id'}, inplace=True)
wikidata.head()

,author,authorLabel,author_id
0,http://www.wikidata.org/entity/Q538109,Ture Nerman,nermatur
1,http://www.wikidata.org/entity/Q4722812,Alfred Hedenstierna,hedensal
2,http://www.wikidata.org/entity/Q5577698,Johan Gustaf Björklund,bjorkljg
3,http://www.wikidata.org/entity/Q6141019,Ludvig Jönsson,jonsslud
4,http://www.wikidata.org/entity/Q4753809,Anders Henrikson,henriand


In [8]:
def aggregator(elements):
    return u",".join(set(elements))

catalogue_authors = catalogue.groupby('author_id', as_index=False).agg(aggregator)

In [9]:
catalogue_authors.head()

,author_id,author,book_id,language,title,type,year
0,,,"trefinska,stud1840,stud1845,nordutid,univhus,v...","fr,dk,no,ee,is,de,p>,it,us,fi,se,fo","Vetenskapen och livet,Boken om Stockholm i ord...","Music,Book,Administrative,Periodical",",1856-1939,1853-1857,1889-1891,1874-1922,1908-..."
1,aakjajep,Jeppe Aakjær,regnebraet,dk,Mit Regnebræt En Selvbibliografi,Book,1919
2,aalljaco,Jacob Aall,norge-1815,no,Erindringer som Bidrag til Norges Historie fra...,Book,1844
3,aaloelev,Ülev Aaloe,svet1979,se,Svensk-estnisk ordbok : Rootsi-eesti sõnaraamat,Book,1979
4,aarestru,Emil Aarestrup,"efterladte,eenrum,aarestrups",dk,"Efterladte Digter,Naar jeg i eenrum færdes,Aar...",Book,",1913,1863"


In [10]:
merged = pd.merge(catalogue_authors, wikidata, on='author_id', how='outer', suffixes=('_runeberg', '_wikidata'))
merged.ix[:, ['author_id', 'author_runeberg', 'authorLabel', 'author_wikidata']]

,author_id,author_runeberg,authorLabel,author_wikidata
0,,,NaN,NaN
1,aakjajep,Jeppe Aakjær,Jeppe Aakjær,http://www.wikidata.org/entity/Q211039
2,aalljaco,Jacob Aall,NaN,NaN
3,aaloelev,Ülev Aaloe,NaN,NaN
4,aarestru,Emil Aarestrup,Emil Aarestrup,http://www.wikidata.org/entity/Q350408
5,aaseniva,Ivar Aasen,NaN,NaN
6,aasvehaa,Haakon Aasvejen,NaN,NaN
7,abergjon,Jon Olof Åberg,NaN,NaN
8,aberssim,Simon Aberstén,NaN,NaN
9,abrahsel,Selma Abrahamsson,NaN,NaN


In [11]:
# How many authors in Danish?
merged.ix[merged.language=='dk', :].shape[0]

122

In [12]:
merged.head()

,author_id,author_runeberg,book_id,language,title,type,year,author_wikidata,authorLabel
0,,,"trefinska,stud1840,stud1845,nordutid,univhus,v...","fr,dk,no,ee,is,de,p>,it,us,fi,se,fo","Vetenskapen och livet,Boken om Stockholm i ord...","Music,Book,Administrative,Periodical",",1856-1939,1853-1857,1889-1891,1874-1922,1908-...",NaN,NaN
1,aakjajep,Jeppe Aakjær,regnebraet,dk,Mit Regnebræt En Selvbibliografi,Book,1919,http://www.wikidata.org/entity/Q211039,Jeppe Aakjær
2,aalljaco,Jacob Aall,norge-1815,no,Erindringer som Bidrag til Norges Historie fra...,Book,1844,NaN,NaN
3,aaloelev,Ülev Aaloe,svet1979,se,Svensk-estnisk ordbok : Rootsi-eesti sõnaraamat,Book,1979,NaN,NaN
4,aarestru,Emil Aarestrup,"efterladte,eenrum,aarestrups",dk,"Efterladte Digter,Naar jeg i eenrum færdes,Aar...",Book,",1913,1863",http://www.wikidata.org/entity/Q350408,Emil Aarestrup
